## Sentiment Analysis

In [1]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |█████████████████               | 309.1 MB 191.7 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 81 kB 103.6 MB/s ta 0:00:01
     |████████████████████████████████| 65 kB 128.8 MB/s eta 0:00:01
     |████████████████████████████████| 14.1 MB 156.0 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 40.7 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 132.1 MB/s eta 0:00:01
     |████████████████████████████████| 4.5 MB 101.9 MB/s eta 0:00:01
     |████████████████████████████████| 5.9 MB 126.3 MB/s eta 0:00:01
     |████████████████████████████████| 438 kB 124.4 MB/s eta 0:00:01
     |████████████████████████████████| 2.4 MB 118.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 120.8 MB/s eta 0:00:01
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1


In [2]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

2022-10-23 18:03:41.818801: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-23 18:03:41.964410: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-23 18:03:42.639145: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/compat/lib.real:/opt/conda/lib/python3.8/site-packages/torch/lib:/opt/conda/lib/python3.8/site-packages/torch_tensorrt/lib:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64


['/device:CPU:0', '/device:GPU:0', '/device:GPU:1']


2022-10-23 18:03:43.570969: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-23 18:03:44.612145: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 33741 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2022-10-23 18:03:44.613667: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:1 with 37221 MB memory:  -> device: 1, name: A100-SXM4-40GB, pci bus id: 0000:0f:00.0, compute capability: 8.0


In [3]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 5.3 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 163 kB 36.5 MB/s eta 0:00:01
     |████████████████████████████████| 7.6 MB 41.3 MB/s eta 0:00:01


In [5]:
import pandas as pd
import numpy as np
import torch
import codecs
import time
from nltk.tokenize import sent_tokenize
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
%cd ..

/workspace/asurion_f22_jw/Asurion-customer-propensity


In [7]:
df = pd.read_csv('data/transcript_may_encoded_100.csv')
# df = pd.read_csv('data/transcript_march_collated_encoded.csv')
df = df.drop(columns='Unnamed: 0')
# df = df[df['accepted_flg'] == True]
df['number_of_words'] = df.encrypted_collated_transcription.apply(lambda x: len(x.split()))
df.head()

,ID,sales_offer_date,accepted_flg,encrypted_collated_transcription,number_of_words
0,4f81e2619aae688a3a814da7f58afdecb9720e9fdd5070...,2022-05-31,False,Thank you for calling [CLIENT] Tech for speaki...,113
1,03bcb2c2dd3e29b8b7ba6c0cf8c7232c8637d6bc73a760...,2022-05-13,False,[CLIENT] tech My name is [NAME]. May I have yo...,454
2,c1eeaa4c1fe8030bb6f0deaa81a13b468001c699586f46...,2022-05-29,False,Hi. Thank you for call. tech you're speaking w...,1674
3,5bf5d93c818534d9edcff8cbb45f28d3e0438cf7ab7850...,2022-05-22,False,Thank you for calling [CLIENT] Tech Coach. My ...,1129
4,c484a81de8c25bcfb9d95f36976cfb425a3d2c7602a93c...,2022-05-21,False,Hi. Thank you for calling [CLIENT] Barcode Coa...,668


In [8]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

In [9]:
def sentence_token_nltk(str):
    sent_tokenize_list = sent_tokenize(str)
    return sent_tokenize_list

In [10]:
# pip install stanza

In [11]:
# import stanza
# def sentence_token_stanza(str):
#     nlp = stanza.Pipeline(lang='en', processors='tokenize')
#     doc = nlp(str)
#     sent_tokenize_list = [sentence.text for sentence in doc.sentences]
#     return sent_tokenize_list

In [14]:
# Import generic wrappers
# from transformers import AutoModel, AutoTokenizer 

from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Define the model repo
myckpt = 'cardiffnlp/twitter-roberta-base-sentiment'
mytokenizer = AutoTokenizer
device = torch.device(str("cuda") if torch.cuda.is_available() else "cpu") 

# Download pytorch model
model = AutoModelForSequenceClassification.from_pretrained(myckpt).to(device)

Downloading: 100%|██████████████████████████████| 747/747 [00:00<00:00, 832kB/s]
Downloading: 100%|███████████████████████████| 499M/499M [00:05<00:00, 97.1MB/s]


In [15]:
def tokenize_inputs(text, checkpoint = myckpt, t=mytokenizer):
    tokenizer = t.from_pretrained(myckpt, is_split_into_words = True, model_max_length=512)
    return tokenizer(text, truncation=True)

In [16]:
def sentiment_analysis(df, text_col:str, length:int):
    '''
    Function to do sentiment analysis by paragraphs into shorter pieces
    '''
    start=time.time()
    for i in df.index:
        iter_start=time.time()
        # break paragraphs into sentences
        sent_list = sentence_token_nltk(df.loc[i, text_col])
        count = 0
        token_list, input_list = [], []
        mask_list, attention_list = [], []
        for sent in sent_list:
            # tokenize sentences
            res = tokenize_inputs(sent)
            tokens, masks = res['input_ids'], res['attention_mask']
            count += len(tokens)

            if count <= length:
                token_list = token_list + tokens
                mask_list = mask_list + masks
            else:    
                input_list.append(token_list)
                attention_list.append(mask_list)
                token_list = tokens
                mask_list = masks
                count=len(tokens)

        input_list.append(token_list)
        attention_list.append(mask_list) 
        for j in range(len(input_list)):      
            # get required padding length
            pad_len = length - len(input_list[j])

            # check if list length satisfies required chunk size
            if pad_len > 0:
                # if padding length is more than 0, we gonna add padding
                input_list[j] = input_list[j] + [0] * pad_len
                attention_list[j] = attention_list[j] + [0] * pad_len

        input_ids = torch.FloatTensor(input_list).to(device)
        attention_mask = torch.FloatTensor(attention_list).to(device)
        input_dict = {'input_ids': input_ids.long(),'attention_mask': attention_mask.int()}
        outputs = model(**input_dict)
#         print(outputs)
        
        probs = torch.nn.functional.softmax(outputs[0], dim=-1)
#         print(probs)
        probs = probs.mean(dim=0)
#         print(probs)
        winner = torch.argmax(probs).item()
#         print(winner)
#         Labels: 0 -> Negative; 1 -> Neutral; 2 -> Positive
        df.loc[i, 'sentiment'] = ['Negative', 'Neutral', 'Positive'][winner]
        df.loc[i, 'runtime'] = round(time.time()-iter_start, 2)
    print(round(time.time()-start, 2), 'seconds')
    return df

In [17]:
df1=sentiment_analysis(df = df, text_col='encrypted_collated_transcription', length=512)

Downloading: 100%|███████████████████████████| 899k/899k [00:00<00:00, 5.30MB/s]
Downloading: 100%|███████████████████████████| 456k/456k [00:00<00:00, 3.17MB/s]
Downloading: 100%|██████████████████████████████| 150/150 [00:00<00:00, 259kB/s]


6441.94 seconds


In [18]:
df1.to_csv('sa_w_twitter.csv', index=0)

In [19]:
df1

,ID,sales_offer_date,accepted_flg,encrypted_collated_transcription,number_of_words,sentiment,runtime
0,4f81e2619aae688a3a814da7f58afdecb9720e9fdd5070...,2022-05-31,False,Thank you for calling [CLIENT] Tech for speaki...,113,Negative,18.34
1,03bcb2c2dd3e29b8b7ba6c0cf8c7232c8637d6bc73a760...,2022-05-13,False,[CLIENT] tech My name is [NAME]. May I have yo...,454,Neutral,48.41
2,c1eeaa4c1fe8030bb6f0deaa81a13b468001c699586f46...,2022-05-29,False,Hi. Thank you for call. tech you're speaking w...,1674,Neutral,171.12
3,5bf5d93c818534d9edcff8cbb45f28d3e0438cf7ab7850...,2022-05-22,False,Thank you for calling [CLIENT] Tech Coach. My ...,1129,Neutral,73.17
4,c484a81de8c25bcfb9d95f36976cfb425a3d2c7602a93c...,2022-05-21,False,Hi. Thank you for calling [CLIENT] Barcode Coa...,668,Neutral,67.75
5,b6716b08ca0380e7b5c7e9b78fcecf5787ccf3d61d1869...,2022-05-03,False,Thank you for calling Coach. This is Sean spea...,2967,Neutral,343.75
6,9f900ab27f1f249ce94d45947421b50ce9b9a85ec0d909...,2022-05-17,True,Thank you for calling [CLIENT] Tech Coach. Ben...,7032,Neutral,723.68
7,143fada0c669611e23e414d405a4f528a2528718a20c00...,2022-05-03,False,Morning. Thanks for [CLIENT] Tech Coach. My na...,2561,Neutral,217.48
8,8cbccda49661f95352e153157f8894c4d2d8e52d28740b...,2022-05-09,False,thank Hello? Hi. Thank you for calling [CLIENT...,1661,Neutral,210.91
9,46f3a9e86aa971a85d0d4cda2a886233d88f9e1da57eff...,2022-05-19,False,Hi. Thank you for [CLIENT] Tech Coach. My name...,536,Neutral,73.96


In [20]:
# df.loc[26, 'encrypted_collated_transcription']

In [21]:
max(df['number_of_words'])

7032